# The Battle of Neighborhood (Week_02) 

### Capstone Project - Canada Burger resturant data Analysis 

# Introduction: # Business Problem

One day I have had a discussion with my friend about where we find burger in Canada city. 

1. I want to look different burger resturants

As part of this project I will try to find the best optimal place in city. Specifically, this report will be targeted to stakeholders interested to find a burger resturant in Canada. Since there are lots of burger resturant in canada, we will try to find places closer to the city. We would also prefer locations as close to city center as possible, assuming that first two conditions are met.I will use my dana analysis knowledge to generate a few most promissing neighborhoods based on this criteria. 

Finally advantages of each area will then be clearly mentioned so that best possible final location can be chosen by stakeholders. 

# Data Description
Based on definition of our problem, factors that will influence our decission are: 
number of existing burger places in neighborhood 
Following data sources will be needed to extract/generate the required information:  
number of burger places and location in every neighborhood will be obtained using Foursquare API

In [1]:
import pandas as pd
import numpy as np
import requests
from matplotlib import cm
from matplotlib import colors 

# Getting the dataset
We will scrape the website https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, to get the details of Toronto neighborhood. The data will be stored into a dataframe.

In [17]:
# define the url
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

# Extract the tables
table_dfs = pd.read_html(url)

# Get the first table
table_df = table_dfs[0]
table_df.head(10)   

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
7,M8A,Not assigned,Not assigned
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"


# Preprocessing of Data
Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [6]:
values = ['Not assigned']
table_df.drop(pd.Index(np.where(table_df['Borough'].isin(values))[0]), inplace = True)
table_df.head()   

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


Total number of rows 

In [8]:
print("There are {} number of rows in the dataframe.".format(table_df.shape[0])) 

There are 103 number of rows in the dataframe.


### Assigning the latitude and longitude for each postal code

We will be using the csv file that has the geographical coordinates of each postal code: http://cocl.us/Geospatial_data . This link gets downloaded with name 'Geospatial_Coordinates.csv'\ We can use pandas read_csv method to read the data into a dataframe.

In [37]:
geodata_df = pd.read_csv('Geospatial_Coordinates.csv')
geodata_df.head()          

FileNotFoundError: [Errno 2] File b'Geospatial.csv' does not exist: b'Geospatial.csv'

In [32]:
table_df = table_df.join(geodata_df.set_index('Postal Code'), on='Postal Code')
table_df.head() 
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_243a7f8563f348d89dbf7163cbfc7f46 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='EzfPwQ0PMZlxuqoQ5aUTR81IQ3o5GkuN2Dw_zxei8inQ',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_243a7f8563f348d89dbf7163cbfc7f46.get_object(Bucket='capstoneprojectweek2-donotdelete-pr-fnu07zxpdvqizz',Key='Geospatial_Coordinates (1).csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_1 = pd.read_csv(body)
df_data_1.head()


NameError: name 'geodata_df' is not defined

### Explore the neighborhoods of Toronto using Foursquare API 

We will be taking borough's that belongs to Toronto area.

In [23]:
toronto_df = table_df.loc[table_df['Borough'].str.contains('Toronto')]
toronto_df.head() 

,Postal Code,Borough,Neighbourhood
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
13,M5B,Downtown Toronto,"Garden District, Ryerson"
22,M5C,Downtown Toronto,St. James Town
30,M4E,East Toronto,The Beaches


Client ID, Secret of Foursquare API 

In [24]:
CLIENT_ID = 'MARBGB0XUSWGYWU2DBN2XNDIC2NJFECHSEP2MY10ZM5QZS2V' # your Foursquare ID
CLIENT_SECRET = 'RKZ0FXERT2WRFR4GRL1WSALH55VGWMXXL2GF0HLNY2GL3RRG' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30 

We will write down a function to get the Burger places in the neighborhoods of Toronto within a radius of 500 meters.

In [25]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&section=food&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results ])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues) 

Now write the code to run the above function on each neighborhood and create a new dataframe called _toronto_Burger restaurants_. 


In [38]:
toronto_Burger restaurants = getNearbyVenues(names=toronto_df['Neighbourhood'],
                                   latitudes=toronto_df['Latitude'],
                                   longitudes=toronto_df['Longitude']
                                  )    

KeyError: 'Latitude'

In [50]:
toronto_Burgerrestaurants.head()  

NameError: name 'toronto_Burgerrestaurants' is not defined

## Visualize the restaurants in Toronto neighborhood 

In [52]:
import folium
map_restaurants = folium.Map(location=[43.6532, -79.3832], zoom_start=11)

# add markers to map
for lat, lng, label in zip(toronto_burger restaurants['Venue Latitude'], toronto_burger restaurants['Venue Longitude'], toronto_restaurants['Venue']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
         popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_burger restaurants)  
    
map_Burger restaurants 

SyntaxError: invalid syntax (<ipython-input-52-5f192e020511>, line 5)

### Let's check how many venues/restaurants were returned for each neighborhood 

In [53]:
toronto_burger restaurants.groupby('Neighborhood').count()['Venue'] 

SyntaxError: invalid syntax (<ipython-input-53-ec06684501db>, line 1)

In [54]:
toronto_burger restaurants_onehot = pd.get_dummies(toronto_Burger restaurants['Venue Category'])
toronto_burger restaurants_onehot.insert(loc=0, column='Neighborhood', value=toronto_Burger restaurants['Neighborhood'])

SyntaxError: invalid syntax (<ipython-input-54-6f9877c85484>, line 1)

In [55]:
toronto_burger restaurants_onehot.head()

SyntaxError: invalid syntax (<ipython-input-55-16e6cac60914>, line 1)

In [56]:
toronto_burger restaurants_grouped = toronto_burger restaurants_onehot.groupby('Neighborhood').mean().reset_index()

NameError: name 'toronto_restaurants_onehot' is not defined

In [57]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues] 

In [58]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_burger restaurants_grouped['Neighborhood']

for ind in np.arange(toronto_burger restaurants_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_restaurants_grouped.iloc[ind, :], num_top_venues)

NameError: name 'toronto_restaurants_grouped' is not defined

# Top 10 Burger restaurants  

In [59]:
neighborhoods_venues_sorted 

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue


## Clustering the neighborhoods 

In [60]:
from sklearn.cluster import KMeans
# set number of clusters
kclusters = 5

toronto_ burger restaurants_grouped_clustering = toronto_burger restaurants_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_burger restaurants_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:20] 

SyntaxError: invalid syntax (<ipython-input-60-1fac09a6ca27>, line 5)

In [61]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')
toronto_merged.head() # check the last columns! 

NameError: name 'kmeans' is not defined

In [62]:
toronto_merged['Cluster Labels'] = toronto_merged['Cluster Labels'].fillna(0)
toronto_merged['Cluster Labels'] = toronto_merged['Cluster Labels'].astype(int)
toronto_merged.head() 

NameError: name 'toronto_merged' is not defined

## Visualize the resulting clusters 

In [64]:
# create map
map_clusters = folium.Map(location=[43.6532, -79.3832], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters 

NameError: name 'folium' is not defined

## Examining each cluster 

In [65]:

toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[2] + list(range(6, toronto_merged.shape[1]))]]

NameError: name 'toronto_merged' is not defined

In [66]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[2] + list(range(6, toronto_merged.shape[1]))]]

NameError: name 'toronto_merged' is not defined

In [67]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[2] + list(range(6, toronto_merged.shape[1]))]]

NameError: name 'toronto_merged' is not defined

In [68]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[2] + list(range(6, toronto_merged.shape[1]))]]

NameError: name 'toronto_merged' is not defined

In [69]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[2] + list(range(6, toronto_merged.shape[1]))]]

NameError: name 'toronto_merged' is not defined

# Conclusion

Based on my finding there are a lot of scope in all clusters except cluster 1. As we observed the analysis was performed on small set of data within 500 meters, it is clear that the data will be better represented by increasing the radius.